### AEP and Area optimisation for HornsRev1 Site

Importing relevant py_wake objects and necessary python packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import math
from sklearn import preprocessing
from scipy.spatial import ConvexHull, convex_hull_plot_2d
#Pywake packages
from py_wake.examples.data.hornsrev1 import V80 # The farm is comprised of 80 V80 turbines which
from py_wake.examples.data.hornsrev1 import Hornsrev1Site #  Horns Rev 1 site,
#from py_wake.examples.data.hornsrev1 import wt_x,wt_y # HornsRev 1 coordinates
from py_wake.examples.data.lillgrund import LillgrundSite
from py_wake.examples.data.lillgrund import wt_x,wt_y
from py_wake.examples.data.lillgrund import SWT23
from py_wake import BastankhahGaussian
from py_wake.deficit_models.gaussian import ZongGaussian 
from py_wake.deficit_models.gaussian import IEA37SimpleBastankhahGaussian, NiayifarGaussian
from py_wake.turbulence_models.stf import STF2017TurbulenceModel
from py_wake.examples.data.iea34_130rwt import IEA34_130_1WT_Surrogate
from py_wake.examples.data.iea37 import IEA37Site
from py_wake.superposition_models import MaxSum
#Topfarm packages
from topfarm.cost_models.cost_model_wrappers import AEPMaxLoadCostModelComponent
from topfarm.cost_models.cost_model_wrappers import CostModelComponent
from topfarm.cost_models.cost_model_wrappers import AEPCostModelComponent
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm import TopFarmProblem
from topfarm.plotting import NoPlot, XYPlotComp
from topfarm.constraint_components.boundary import CircleBoundaryConstraint
from topfarm.constraint_components.boundary import XYBoundaryConstraint
from topfarm.constraint_components.spacing import SpacingConstraint
from topfarm.cost_models.py_wake_wrapper import PyWakeAEPCostModelComponent

### Function to create a Square windfarm layout 

In [ ]:
def get_turbines(n_wt):
    '''
    This function generates a ficitonal squared wind farm layout with user input no of turbines is a squar number.
    Parameters
    ----------
    n_wt(int) : Number of wind turbines in square numbers
    Returns
    -------
    wt_xrt(list) : x-coordinates after rotation/translation/rot_trans
    wt_yrt(list) : y-coordinates after rotation/translation/rot_trans
    '''
    wt_x = [] # List to store initial turbine coordinates 
    wt_y = [] 
    row=int(n_wt/np.sqrt(n_wt)) # defining number of turbines in each row
    # Generate initial layout for the given number of turbines
    for i in range(row):
        for j in range(row):
            nt_x = (j) # implements spacing in xcoordinates, coloumn wise increment
            nt_y = (i) # Implements spacing in ycoordinates
            wt_x.append(nt_x)
            wt_y.append(nt_y)
    coordinates = np.array([wt_x,wt_y])
    plt.figure()
    plt.plot(wt_x,wt_y,'.k')
    plt.xlabel('Distance in x direction [m]')
    plt.ylabel('Distance in y direction [m]')
    plt.title('Fictional wind farm layout')
    return wt_x,wt_y
get_turbines(16)

### Defined a Pywake Site, Wind Turbine, Wake deficit and Turbulence models
##### Site: A Uniform weibull site with ambient turbulent intesity of 0.1, Vref reference wind speed height at 80mts, turbine coordinates are defined.
##### Wind turbine model: An IEA3.4MW load surrogate model is chosen here.
##### Wind farm model: IEA 3.4MW Wind trubine, BastankhahGaussian wake deficit model, Stefan Frandsen turbulence model, Maxsum superposition model.

In [ ]:
n_wt = 16 #Number turbines in wind farm
site = Hornsrev1Site() # Defines Wind farm site
# wt = V80() # Defines Wind Turbine
# windFarmModel = BastankhahGaussian(site, wt) # Flow Model
wt = IEA34_130_1WT_Surrogate()
windFarmModel = BastankhahGaussian(site, wt, turbulenceModel=STF2017TurbulenceModel(addedTurbulenceSuperpositionModel=MaxSum()))
# Plot wind distribution
site.plot_wd_distribution(n_wd=12)

In [ ]:
# Turbine thrust and power curves
ws = np.arange(5,25)
plt.xlabel('Wind speed [m/s]')
plt.ylabel('Power [kW]')
plt.plot(ws, wt.power(ws,TI_eff =0.1)*1e-3,'.-', label='Power curve')
plt.plot(ws, wt.ct(ws,TI_eff =0.1),'.-', label='Thrust')
plt.legend(loc=1)

In [ ]:
# Power and Thrust curve plots
fig, ax1 = plt.subplots() 
  
ax1.set_xlabel('Wind Speeds in m/s') 
ax1.set_ylabel('Power in Kw', color = 'red') 
ax1.plot(ws, wt.power(ws,TI_eff =0.1)*1e-3,'.-',color = 'red') 
ax1.tick_params(axis ='y', labelcolor = 'red') 
  
# Adding Twin Axes

ax2 = ax1.twinx() 
  
ax2.set_ylabel('Thrust coefficient', color = 'blue') 
ax2.plot(ws, wt.ct(ws,TI_eff =0.1),'.-',color = 'blue') 
ax2.tick_params(axis ='y', labelcolor = 'blue') 
plt.title('Power and Thrust curve of IEA3.4MW turbine')
 
# Show plot

plt.show()

In [ ]:
# from py_wake.superposition_models import MaxSum
# D=130
# plt.figure(figsize=(10,4))
# plt.xlabel('Downwind distance [m]')
# plt.ylabel('Crosswind distance[m]')
# plt.title('Bastankhah Guassian wake deficit model')
# windFarmModel([0,4*D],[0,0],wd=260,ws=12).flow_map().plot_wake_map(levels=np.arange(1,13))

In [ ]:
# windFarmModel([0,4*D],[0,0],wd=260,ws=12)

In [ ]:
# # plot load curves
# u = np.arange(3, 28, 1)
# sensors = wt.loadFunction.output_keys
# axes = [plt.figure().gca() for _ in sensors]
# for ti in [0.01, .05, .1, .3]:
#     loads = wt.loads(u, TI_eff=ti)
#     for ax, l in zip(axes, loads):
#         ax.plot(u, l, label=f'TI={ti}')
# # for alpha in [-0.09, .1, .3, .49]:
# #     loads = wt.loads(u, TI_eff=.1, Alpha=alpha)
# #     for ax, l in zip(axes, loads):
# #         ax.plot(u, l, '--', label=f'Alpha={alpha}')
# for ax, s in zip(axes, sensors):
#     ax.set_title(s)
#     ax.legend()

In [ ]:
# # plot loads as function of wd and ws
# plt.figure()
# site = Hornsrev1Site()
# x, y = [0,4*D],[0,0]
# load_wd_averaged = windFarmModel([0,4*D],[0,0],wd=260,ws=12).loads(normalize_probabilities=True, method='OneWT_WDAvg')
# loads = windFarmModel([0,4*D],[0,0],wd=260,ws=12).loads(normalize_probabilities=True, method='OneWT')
# loads.DEL.isel(sensor=0, wt=0).plot()

# for s in load_wd_averaged.sensor:
#     print(s.item(), load_wd_averaged.LDEL.sel(sensor=s, wt=0).item(), loads.LDEL.sel(sensor=s, wt=0).item())
# plt.show()

In [ ]:
# plt.figure(figsize=(10,4))
# plt.xlabel('Downwind distance [m]')
# plt.ylabel('Crosswind distance[m]')
# plt.title('Bastankhah Gaussian deficit')
# windFarmModel([0,4*D],[0,0],wd=260,ws=12).flow_map().plot_ti_map()

In [ ]:
# TI_model = BastankhahGaussian(site, wt, turbulenceModel=STF2017TurbulenceModel(addedTurbulenceSuperpositionModel=MaxSum()))

In [ ]:
# TI_model([0,4*D],[0,0],wd=260,ws=12)

In [ ]:
# # methods to plot turbulence map, used below to visualize and compare the deficit models

# def plot_turb_map(turbulenceModel):
#     xy = np.linspace(-200,500,200), np.linspace(-200,200,200)
#     X,Y,deficit = _map(turbulenceModel.calc_added_turbulence, xy)
#     c = plt.contourf(X,Y,deficit, levels=100, cmap='Blues')
#     plt.colorbar(c, label="Added turbulence intensity [-]")
#     plt.plot([0,0],[-1/2,1/2],'k')
#     plt.ylabel("Crosswind distance [y/D]")
#     plt.xlabel("downwind distance [x/D]")

# from py_wake.turbulence_models import STF2017TurbulenceModel
# plot_turb_map(STF2017TurbulenceModel())

##### A new functionality has been added to the previously defined square wind farm layout, allowing the turbines to be rotated and translated, thereby optimizing the layout.
##### The area of the layout is calculated by multiplying the Euclidean length and width of the final layout.
##### The annual energy production of the wind farm layout is calculated at each iteration.

In [ ]:
def get_AEP(tr):
    '''
    This function generates a ficitonal squared wind farm layout for a user input no of turbines
    and tries to change each turbine location either by rotation/ translation or both.
    Parameters
    ----------
    tr(list) : A list with x,y spacing variables and theta.
    tr[0]-sx(int) : Initial Spacing between the turbines in x-direction, mts
    tr[1]-sy(int) : Initial Spacing between the turbines in y-direction, mts
    tr[2]-theta(int) : For rotation, input theta in degrees
    Returns
    -------
    aep_final(float) : Annual Energy production of final wind farm layout in Gwh
    final_area(float) : Final layout area in m2.
    wt_xrt(list) : X-coordinates of turbine positions
    wt_yrt(list) : y-coordinates of turbine positions
    '''
    wt_rtx = []
    wt_rty = [] # List to store turbine coordinates after translation and rotation
    sx = tr[0] # Spacing between turbines in x-direction
    sy = tr[1] # Spacing between turbines in y-direction
    theta = tr[2] # Theta in degrees
    row=int(n_wt/np.sqrt(n_wt)) # defining number of turbines in each row
     # Translate and rotate the initial layout
    for i in range(row):
        for j in range(row):
            wt_tx = (j)*sx # Increases spacing from intial coordinates
            wt_ty = (i)*sy 
            wt_rtx.append(wt_tx)
            wt_rty.append(wt_ty)
    coordinates_rt = np.array([wt_rtx,wt_rty])
    # # Implement boundary constraint
    # if (theta<=45) :
    #     center = [max(coordinates_rt.T[:,0])/2,max(coordinates_rt.T[:,1])/2] # Circle center
    #     r = math.dist([0,0],center)*1.5 # Radius of circle
    # else:
    #     center = [-(max(coordinates_rt.T[:,0])/2),(max(coordinates_rt.T[:,1])/2)]
    #     r = math.dist([0,0],center)*1.5
    theta=np.deg2rad(theta) # Converts theta from degrees to radians
    rotation = np.array([[np.cos(theta),-np.sin(theta)],
                            [np.sin(theta),np.cos(theta)]])
    final_coordinates = np.dot(rotation,coordinates_rt) # x,y-coordinates after rotation
    wt_xrt = final_coordinates.T[:,0]
    wt_yrt = final_coordinates.T[:,1]
    # Plot initial and final layout
    # M = 1000
    # angle = np.exp(1j * 2 * np.pi / M)
    # angles = np.cumprod(np.ones(M + 1) * angle)
    # x, y = np.real(angles), np.imag(angles) 
    # X = center[0]+r * x
    # Y = center[1]+r * y
    plt.plot(wt_xrt,wt_yrt,'.b')
    # plt.plot(X, Y)
    #Calculate area of final layout
    v1 = np.array([wt_xrt[0], wt_yrt[0]]) # Vertices of final layout
    v2 = np.array([wt_xrt[row-1], wt_yrt[row-1]])
    v3 = np.array([wt_xrt[-1], wt_yrt[-1]])
    v4 = np.array([wt_xrt[-1], wt_yrt[-1]])
    l = math.dist(v1,v2) # Distance between vertices
    b = math.dist(v2,v3)
    final_area = l*b
    #Calculate Annual Energy Production of final layout
    # wsp = np.asarray([4, 24])
    # wdir = np.arange(0,360,1)
    res = windFarmModel(wt_xrt,wt_yrt)
    aep_final = res.aep().sum()
    # # Loop to check if any of the turbine crossing boundary
    # dist_v2c = math.dist(center,v2)
    # dist_v3c = math.dist(center,v3)
    # dist_v4c = math.dist(center,v4)
    # if ((r>dist_v2c)&(r>dist_v3c)&(r>dist_v4c)):
    #     rmax = 0.5
    #     rmax = np.float128(rmax)
    # else:
    #     rmax = 1
    #     rmax = np.float128(rmax)
    return aep_final,final_area,wt_xrt,wt_yrt,res
def get_optimize(tr):
    '''
    This function generates a ficitonal squared wind farm layout for a user input no of turbines
    and tries to change each turbine location either by rotation/ translation or both.
    Parameters
    ----------
    tr[0]-sx(int) : Initial Spacing between the turbines in x-direction, mts
    tr[1]-sy(int) : Initial Spacing between the turbines in y-direction, mts
    tr[2]-theta(int) : For rotation, input theta in degrees
    Returns
    -------
    aep_final : Annual Energy production of final wind farm layout in Gwh
    '''
    aep_final,final_area,wt_xrt,wt_yrt,res = get_AEP(tr)
    return aep_final
def get_area(tr):
    '''
    This function generates a ficitonal squared wind farm layout for a user input no of turbines
    and tries to change each turbine location either by rotation/ translation or both.
    Parameters
    ----------
    tr[0]-sx(int) : Initial Spacing between the turbines in x-direction, mts
    tr[1]-sy(int) : Initial Spacing between the turbines in y-direction, mts
    tr[2]-theta(int) : For rotation, input theta in degrees
    Returns
    -------
    finalarea, aep_final[list] : Returns a list of Layout AEP and corresponding Area
    '''
    aep_final,final_area,wt_xrt,wt_yrt,res = get_AEP(tr)
    return [final_area,aep_final]
def get_coord(tr):
    '''
    This function generates a ficitonal squared wind farm layout for a user input no of turbines
    and tries to change each turbine location either by rotation/ translation or both.
    Parameters
    ----------
    tr[0]-sx(int) : Initial Spacing between the turbines in x-direction, mts
    tr[1]-sy(int) : Initial Spacing between the turbines in y-direction, mts
    tr[2]-theta(int) : For rotation, input theta in degrees
    Returns
    -------
    wt_xrt(list) : X-coordinates of turbine positions
    wt_yrt(list) : y-coordinates of turbine positions
    '''
    aep_final,final_area,wt_xrt,wt_yrt,res = get_AEP(tr)
    return wt_xrt,wt_yrt
def get_loads(tr):
    aep_final,final_area,wt_xrt,wt_yrt,res = get_AEP(tr)
    loads = res.loads('OneWT')['LDEL'].values
    return [aep_final,final_area,loads]
def get_cap_density(tr):
    aep_final,final_area,wt_xrt,wt_yrt,res = get_AEP(tr)
    aep_final = aep_final*1e3 # GwH to MwH
    final_area = final_area*1e-6 # m2 to Km2
    cd = aep_final/final_area
    loads = res.loads('OneWT')['LDEL'].values
    return [cd,aep_final,final_area,loads]

### Step 1 : Define TopFarm problem to maximize AEP with lower and upper bound spacing constraints

- n_wt - No of wind turbines 
- D - Diameter of IEA3.4Mw wind turbine
- Solver - Sequential Least-Squares programming algorithm (SLSQP) - Gradient based
- maxiter - Maximum number of iterations
- tol - Optimizer tolerance
- tr - [spacing in x, spacing in y, Theta in degrees]
- lb - lower bounds
- ub - upper bounds
- ap - initial value for aep

In [ ]:
#TopFarm problem to maximize aep
n_wt = 16 # 
D = 130 # Diameter of 3.4Mw turbine
maxiter = 500
tol = 1e-7
tr = [4*D,4*D,0] # Initial conditions 
step = 1e-3
lb = [3*D,3*D,0] # Lower bound 
ub = [8*D,8*D,90] # Upper bound
ap=0 
aep_comp1 = CostModelComponent(input_keys=[('tr',tr)],
                              n_wt=n_wt,
                              cost_function=get_optimize,
                              objective=True,
                              maximize=True, 
                              step={'tr': step},
                              output_keys=[('aep_final',ap)])
                              
problem1 = TopFarmProblem(design_vars={'tr':(tr,lb,ub)},
                          cost_comp=aep_comp1,
                          driver=EasyScipyOptimizeDriver(optimizer='SLSQP',maxiter=maxiter,tol=tol))

In [ ]:
cost1, state1, recorder1 = problem1.optimize()

In [ ]:
get_loads(state1['tr'])

In [ ]:
# recorder1.save('AEP_calc_seq1')

In [ ]:
state = state1['tr']
print(state)
get_optimize(state)

In [ ]:
# Get coordinates of initial and optimised layout
wt_aepx_opt,wt_aepy_opt = get_coord(state) # Optimised layout coordinates
wt_aepx_int,wt_aepy_int = get_coord(tr) # Initial layout coordinates

In [ ]:
def get_con_area(x,y):
    coordinates = np.zeros((len(x),2))
    for i in range (len(coordinates)):
        coordinates[i] = [x[i],y[i]]
    points = coordinates  # Wind farm points
    hull = ConvexHull(points)
    vert_indx = np.sort(hull.vertices)
    con_vert = coordinates[vert_indx]
    area = hull.volume
    return area
wt_arx,wt_ary = get_coord(state)
get_con_area(wt_arx,wt_ary)

In [ ]:
# get_area(state)

In [ ]:
# Plots for design variables and AEP
plt.figure()
plt.plot(recorder1['aep_final'],label='AEP in Gwh')
plt.xlabel('No of Iterations')
plt.ylabel('AEP [GWh]')
plt.title(' Variation of AEP during optimization')
plt.legend()

In [ ]:
xcoord = recorder1['tr']
plt.plot(xcoord[:,0],label='x-coord')
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in x-direction [mts]')
plt.title('Change in spacing- horizontal direction')
plt.legend()

In [ ]:
plt.plot(xcoord[:,1],'m',label='y-coord',)
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in y-direction [mtrs]')
plt.title('Change in spacing -vertical direction')
plt.legend()

In [ ]:
plt.plot(xcoord[:,2],'g',label='Theta')
plt.xlabel('No of Iterations')
plt.ylabel('Theta [deg]')
plt.title('Change in Theta')
plt.legend()

In [ ]:
plt.figure()
plt.plot(xcoord[:,0],label='x-coord in mts')
plt.plot(xcoord[:,1],'m',label='y-coord in mts')
plt.plot(xcoord[:,2],label='Theta in Deg')
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in x and y [mts] and Theta [Deg]')
plt.legend()
plt.show()

#### Comparison of optimized layout with initial layout after step1

In [ ]:
plt.figure()
plt.scatter(wt_aepx_opt,wt_aepy_opt,label='optimized layout',marker='*')
# plt.scatter(farm_lx,farm_ly,label='Farm Boundary',marker='o')
# plt.scatter(bound_lx,bound_ly,label='Initial Boundary')
# plt.scatter(recording3['x'][1],recording3['y'][1],label='x,y 2nd', marker='*')
# plt.scatter(recording3['x'][2],recording3['y'][2],label='x,y 3rd', marker='_')
plt.scatter(wt_aepx_int,wt_aepy_int,label='Iniital layout', marker='o')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.title('Wind farm layout during AEP optimization')
#plt.plot(wt_nx,wt_ny,'2b')
#plt.plot(wt_areax,wt_areay,'2g')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
aep_res = windFarmModel(wt_aepx_int,wt_aepy_int)
flow_map = aep_res.flow_map(ws = 10, wd=260)
flow_map.plot_wake_map()
plt.title('Flow map of Initial wind farm layout, Ambient TI=0.1,WS=10m/s')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')

#### Flowmap of Optimized layout

In [ ]:
plt.figure(figsize=(8,6))
aep_res = windFarmModel(wt_aepx_opt,wt_aepy_opt)
flow_map = aep_res.flow_map(ws = 10, wd=260)
flow_map.plot_wake_map()
plt.title('Flow map of Optimized wind farm layout, Ambient TI=0.1,WS=10m/s')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')

### Step2: Define TopFarm problem to minimize Area under lower and upper bound spacing constraints
 
- n_wt - No of wind turbines 
- D - Diameter of IEA3.4Mw wind turbine
- Solver - SLSQP
- maxiter - Maximum number of iterations
- tol - Optimizer tolerance
- tr - [spacing in x, spacing in y, Theta in degrees]
- lb - lower bounds
- ub - upper bounds
- ap - initial value for aep
- max_aep - maximum aep is set to 98% of optimal AEP

In [ ]:
# Topfarm problem to minimize area
n_wt=16
D=130
maxiter=500
tol=1e-7
tr = state # Initial condition starting from optimal values of AEP optimization
step = 1e-3
lb = [3*D,3*D,0]
ub = [8*D,8*D,90]
ap=0
max_aep = -0.98*cost1
print(max_aep)
aep_comp2 = CostModelComponent(input_keys=[('tr',tr)],
                                n_wt=n_wt, cost_function=get_area,
                                objective=True, step={'tr': step},
                                output_keys=[('final_area',0),('aep_final',ap)])
problem2 = TopFarmProblem(design_vars= {'tr':(tr,lb,ub)},
                            post_constraints=[('aep_final',{'lower':max_aep})],
                            cost_comp=aep_comp2,
                            driver=EasyScipyOptimizeDriver(optimizer='SLSQP', maxiter=maxiter,tol=tol),
                            expected_cost=1)

In [ ]:
cost2, state2, recorder2 = problem2.optimize()

In [ ]:
# recorder2.save('Area_calc_seq2')

In [ ]:
state_ar = state2['tr']
print(state_ar)
get_area(state_ar)

In [ ]:
# Get coordinates of initial and optimised layout
wt_areax_opt,wt_areay_opt = get_coord(state_ar) # Optimised layout coordinates
wt_areax_int,wt_areay_int = get_coord(state) # Initial layout coordinates

In [ ]:
plt.figure()
plt.scatter(wt_areax_opt,wt_areay_opt,label='optimized layout',marker='*')
# plt.scatter(farm_lx,farm_ly,label='Farm Boundary',marker='o')
# plt.scatter(bound_lx,bound_ly,label='Initial Boundary')
# plt.scatter(recording3['x'][1],recording3['y'][1],label='x,y 2nd', marker='*')
# plt.scatter(recording3['x'][2],recording3['y'][2],label='x,y 3rd', marker='_')
plt.scatter(wt_areax_int,wt_areay_int,label='Iniital layout', marker='o')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.title('Wind farm layout during Area optimization')
#plt.plot(wt_nx,wt_ny,'2b')
#plt.plot(wt_areax,wt_areay,'2g')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
area_res = windFarmModel(wt_areax_opt,wt_areay_opt)
flow_map = area_res.flow_map(ws = 10, wd=260)
flow_map.plot_wake_map()
plt.title('Flow map of Optimized wind farm layout, Ambient TI=0.1,WS=10m/s')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.show()

In [ ]:
# Plots for design variables and AEP
plt.plot(recorder2['aep_final'],label='AEP in Gwh')
plt.plot(plt.xlim(),[max_aep,max_aep], label='Minimum AEP' )
plt.xlabel('No of Iterations')
plt.ylabel('AEP [GWh]')
plt.title('Variation in AEP during Area Optimization')
plt.legend()

In [ ]:
# Plots for design variables and AEP
plt.plot(recorder2['final_area'],'k',label='Area in m2')
plt.title('Variation in Area during Area optimization')
plt.xlabel('No of Iterations')
plt.ylabel('Area [m2]')
plt.legend()

In [ ]:
coord = recorder2['tr']
plt.plot(coord[:,0],label='x-coord')
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in x-direction [mts]')
plt.title('Change in spacing for horizontal direction')
plt.legend()

In [ ]:
plt.plot(coord[:,1],'m',label='y-coord',)
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in y-direction [m]')
plt.title('Change in spacing for vertical direction')
plt.legend()

In [ ]:
plt.plot(coord[:,2],'k',label='Theta',)
plt.xlabel('No of Iterations')
plt.ylabel('Theta [Deg]')
plt.title('Change in Theta')
plt.legend()

In [ ]:
plt.plot(coord[:,0],label='y-coord')
plt.plot(coord[:,1],label='x-coord')
plt.plot(coord[:,2],label='Theta in Deg')
plt.xlabel('No fo Iterations')
plt.ylabel('Coordinates and Theta')
plt.legend()

In [ ]:
Area_opt_AEP = [303.10780531,297.04564918,294.04,291.01,287.98]
Area_opt_Area = [5153527.946294739,5014358.82992715,2856048.9916359526,2135221.387851181,1760763.5648305602]
plt.plot(Area_opt_AEP,Area_opt_Area)
plt.xlabel('Maximum AEP [GWh]')
plt.ylabel('Optimised Area [m2]')
plt.title('Pareto Front')

In [ ]:
trco = state_ar
x,y = get_coord(trco)

### Calculation of Maximum Loads
- i - No of wind turbines
- trco - Initial condition - from optimal values of Area optimization
- load factor - 3% upscaled

In [ ]:
i = 16
# trco = state_ar # Initial conditions to start the optimisation
# x,y = get_coord(trco) # Turbine positions from initial conditions
# wsp = np.asarray([4, 24])
# wdir = np.arange(0,360,12)
# k = wsp.size
# l = wdir.size
load_fact = 1.03
# simulationResult = windFarmModel(x,y,wd=wdir, ws=wsp)
# nom_loads = simulationResult.loads('OneWT')['LDEL'].values
# max_loads = (nom_loads * load_fact)
aep, area, nom_loads = get_loads(trco)
max_loads = (nom_loads*load_fact)
s = nom_loads.shape[0]
load_signals = ['del_blade_flap', 'del_blade_edge', 'del_tower_bottom_fa',
                'del_tower_bottom_ss', 'del_tower_top_torsion']
output_loads = np.zeros((5,16))
# print(max_loads)


In [ ]:
# Function to calculate loads of a wind turbines
# def get_Load(tr):
#     # tr = [sx,sy,theta]
#     aep_final,final_area,wt_xrt,wt_yrt = get_AEP(tr)
#     loads = windFarmModel(wt_xrt,wt_yrt,wd=wdir, ws=wsp).loads('OneWT')['LDEL'].values
#     return [aep_final,final_area,loads]

### Step3: Define TopFarm problem to maximize AEP under fatigue load constraints
 
- n_wt - No of wind turbines 
- D - Diameter of IEA3.4Mw wind turbine
- Solver - SLSQP
- maxiter - Maximum number of iterations
- tol - Optimizer tolerance
- tr - [spacing in x, spacing in y, Theta in degrees]
- lb - lower bounds
- ub - upper bounds
- max area - 5% upscale. This metric is included to allow the solver to refine the design space to maximize AEP.

In [ ]:
# Declare optimizer values for Load constraints
n_wt = 16
D = 130
step = 1e-3
tol = 1e-7
trco = state_ar #Initial conditions - optimal values from Area optimization
lb = [3*D,3*D,0]
ub = [8*D,8*D,90]
#min_aep = -0.97*cost1
#print(min_aep)
max_area = recorder2['final_area'][-1]
max_area = max_area*1.05
print(max_area)
# Setup aep/cost component
aep_comp3 = CostModelComponent(input_keys=[('tr',trco)],
                                          n_wt = n_wt,
                                          cost_function = get_loads,
                                          step = {'tr':step},
                                          objective=True,
                                          maximize=True,
                                          output_keys=[ ('aep_final',0),('final_area', 0),('loads', output_loads)]
                                          )

In [ ]:
# Setup topfarm problem for aep maximization with load constraints
problem3 = TopFarmProblem(design_vars={'tr':(trco,lb,ub)},
                          post_constraints=[('final_area',{'upper': max_area}),
                           ('loads',{'upper':max_loads}) ], 
                          cost_comp=aep_comp3,
                          driver=EasyScipyOptimizeDriver(optimizer='SLSQP', maxiter=maxiter,tol=tol),
                          expected_cost=1
                            ) 

In [ ]:
max_area

In [ ]:
cost3,state3,recorder3 = problem3.optimize()

In [ ]:
# recorder3.save('Aep_loads_calc_seq3')

In [ ]:
state_lc = state3['tr']
print(state_lc)
get_loads(state_lc)

In [ ]:
plt.figure()
plt.plot(recorder3['aep_final'], label ='AEP in Gwh')
plt.xlabel('No of iterations')
plt.ylabel('AEP [Gwh]')
plt.title('AEP during AEP maximization under load constraints')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(recorder3['final_area'],label='optimised area in m2')
plt.plot(plt.xlim(),[max_area,max_area], label='Maximum Area in m2' )
plt.xlabel('No of iterations')
plt.ylabel('Area [m2]')
plt.title('Area during AEP maximization under load constraints')
plt.legend()
plt.show()

In [ ]:
# Get coordinates of initial and optimised layout
wt_aepldx_opt,wt_aepldy_opt = get_coord(state_lc) # Optimised layout coordinates
wt_aepldx_int,wt_aepldy_int = get_coord(state_ar) # Initial layout coordinates

In [ ]:
plt.figure()
plt.scatter(wt_aepldx_opt,wt_aepldy_opt,label='optimized layout',marker='*')
# plt.scatter(farm_lx,farm_ly,label='Farm Boundary',marker='o')
# plt.scatter(bound_lx,bound_ly,label='Initial Boundary')
# plt.scatter(recording3['x'][1],recording3['y'][1],label='x,y 2nd', marker='*')
# plt.scatter(recording3['x'][2],recording3['y'][2],label='x,y 3rd', marker='_')
plt.scatter(wt_aepldx_int,wt_aepldy_int,label='Iniital layout', marker='o')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.title('Wind farm layout during AEP optimization under load constraints')
#plt.plot(wt_nx,wt_ny,'2b')
#plt.plot(wt_areax,wt_areay,'2g')
plt.legend()
plt.show()

In [ ]:
n_i = recorder3['counter'].size
loads = recorder3['loads'].reshape((n_i,s,n_wt))
wt = 0
for n, ls in enumerate(load_signals):
  plt.plot(loads[:,n,wt],label='optimised loads')
  plt.title(ls+f' turbine {wt}')
  plt.plot([0, n_i], [max_loads[n, wt], max_loads[n, wt]],label='Maximum loads')
  plt.xlabel("No of iterations")
  plt.ylabel("Loads")
  plt.legend()
  plt.show()

In [ ]:
# Declare optimizer values for energy density with load constraints
n_wt = 16
D = 130
step = 1e-3
tol = 1e-6
trco = state_lc #Initial conditions - optimal values from loads optimization
lb = [3*D,3*D,0]
ub = [8*D,8*D,90]
min_aep = -cost3*1e3*0.98
print(min_aep)
max_area = recorder2['final_area'][-1]
max_area = max_area*1e-6*1.05
print(max_area)
# Setup aep/cost component
aep_comp4 = CostModelComponent(input_keys=[('tr',trco)],
                                          n_wt = n_wt,
                                          cost_function = get_cap_density,
                                          step = {'tr':step},
                                          objective=True,
                                          maximize=True,
                                          output_keys=[ ('cd',0),('aep_final',0),('final_area', 0),('loads', output_loads)]
                                          )

In [ ]:
# Setup topfarm problem for aep maximization with load constraints
problem4 = TopFarmProblem(design_vars={'tr':(trco,lb,ub)},
                          post_constraints=[('aep_final',{'lower': min_aep}),('final_area',{'upper': max_area}),
                           ('loads',{'upper':max_loads}) ], 
                          cost_comp=aep_comp4,
                          driver=EasyScipyOptimizeDriver(optimizer='SLSQP', maxiter=maxiter,tol=tol),
                          expected_cost=1
                            )


In [ ]:
cost4,state4,recorder4 = problem4.optimize()

In [ ]:
# recorder4.save('Ed_calc_seq4')

In [ ]:
state_cd = state4['tr']
print(state_cd)
get_cap_density(state_cd)

In [ ]:
plt.plot(recorder4['aep_final'],label='Optimised_AEP')
plt.plot(plt.xlim(),[min_aep,min_aep], label='Minimum AEP in Mwh' )
plt.xlabel('No of iterations')
plt.ylabel('AEP [Mwh]')
plt.title('Variation of AEP during Energy Density optimization')
plt.legend()
plt.show()

In [ ]:
plt.plot((recorder4['cd']/8760),label='Energy_denisty')
plt.xlabel('No of iterations')
plt.ylabel('Energy Density [Mw/Km2]')
plt.title('Variation of Energy Density during Energy Density optimization')
plt.legend()
plt.show()

In [ ]:
plt.plot(recorder4['final_area'],label='Area in Km2')
plt.xlabel('No of iterations')
plt.ylabel('Area [km2]')
plt.title('Variation of Areaduring Energy Density optimization')
plt.legend()
plt.show()

In [ ]:
coord=recorder4['tr']
plt.plot(coord[:,0],'b',label='x-coord',)
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in x-direction [m]')
plt.title('Change in spacing for horizontal direction')
plt.legend()

In [ ]:
plt.plot(coord[:,1],'m',label='y-coord',)
plt.xlabel('No of Iterations')
plt.ylabel('Spacing in y-direction [m]')
plt.title('Change in spacing for vertical direction')
plt.legend()

In [ ]:
plt.plot(coord[:,2],'k',label='Theta',)
plt.xlabel('No of Iterations')
plt.ylabel('Theta [deg]')
plt.title('Change in Theta')
plt.legend()

In [ ]:
# Get coordinates of initial and optimised layout
wt_edx_opt,wt_edy_opt = get_coord(state_cd) # Optimised layout coordinates
wt_edx_int,wt_edy_int = get_coord(state_lc) # Initial layout coordinates

In [ ]:
plt.figure()
plt.scatter(wt_edx_opt,wt_edy_opt,label='optimized layout',marker='*')
# plt.scatter(farm_lx,farm_ly,label='Farm Boundary',marker='o')
# plt.scatter(bound_lx,bound_ly,label='Initial Boundary')
# plt.scatter(recording3['x'][1],recording3['y'][1],label='x,y 2nd', marker='*')
# plt.scatter(recording3['x'][2],recording3['y'][2],label='x,y 3rd', marker='_')
plt.scatter(wt_aepx_opt,wt_aepy_opt,label='Iniital layout', marker='o')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.title('Comparison of wind farm layout after first and last optimization')
#plt.plot(wt_nx,wt_ny,'2b')
#plt.plot(wt_areax,wt_areay,'2g')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
area_res = windFarmModel(wt_aepldx_opt,wt_aepldy_opt)
flow_map = area_res.flow_map(ws = 10, wd=260).plot_ti_map()
# flow_map.plot_wake_map()
plt.title('Flow map of Initial wind farm layout, Ambient TI=0.1,WS=10m/s')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
ed_res = windFarmModel(wt_edx_opt,wt_edy_opt)
flow_map = ed_res.flow_map(ws = 10, wd=260).plot_ti_map()
# flow_map.plot_wake_map()
plt.title('Flow map of Optimized wind farm layout, Ambient TI=0.1,WS=10m/s')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.show()

In [ ]:
n_i = recorder4['counter'].size
loads = recorder4['loads'].reshape((n_i,s,n_wt))
wt = 0
for n, ls in enumerate(load_signals):
  plt.plot(loads[:,n,wt],label='optimised loads')
  plt.title(ls+f' turbine {wt}')
  plt.plot([0, n_i], [max_loads[n, wt], max_loads[n, wt]],label='Maximum loads')
  plt.xlabel("No of iterations")
  plt.ylabel("Loads")
  plt.legend()
  plt.show()

In [ ]:
wt_xaep,wt_yaep = get_coord(state)
wt_xarea,wt_yarea = get_coord(state_ar)
wt_xload,wt_yload = get_coord(state_lc)
wt_xcd,wt_ycd = get_coord(state_cd)
aep1 = get_optimize(state)
#aep1['AEP']

In [ ]:
get_area(state_cd)

In [ ]:
get_area(state_lc)

In [ ]:
aep1 = np.array(aep1)
aep1 = np.round(aep1,2)
print(aep1)
area1 = get_area(state)
area1 = np.array(area1)
area1 = np.round(area1[0]*1e-6,2)
aep_opt = get_loads(state)
aep_opt_loads = aep_opt[2]
aep_opt_tbfa = aep_opt_loads[2]
print(aep_opt_tbfa)

In [ ]:
AEP = [303.1078,297.0456,297.629,291.677]
plt.figure()
plt.plot(AEP,'--bo',label='AEP[Gwh]')
# plt.plot(AEP[1],'--bo',label='Minimize Area')
# plt.plot(AEP[2],'--bo',label='AEP under loads')
# plt.plot(AEP[3],'--bo',label='Minimize Energy density')
plt.xticks([0,1,2,3,],['Maximize AEP','Minimize Area','AEP under loads','Maximize Energy density'])
plt.ylabel('AEP in Gwh')
plt.title('AEP generated for multiple objectives')
plt.legend()

In [ ]:
Area = [9734399.99,5014358.8299,5265076.77,3421825.016]
plt.figure()
plt.plot(AEP,'--go',label='Area[m2]')
# plt.plot(AEP[1],'--bo',label='Minimize Area')
# plt.plot(AEP[2],'--bo',label='AEP under loads')
# plt.plot(AEP[3],'--bo',label='Minimize Energy density')
plt.xticks([0,1,2,3,],['Maximize AEP','Minimize Area','AEP under loads','Maximize Energy density'])
plt.ylabel('Area in m2')
plt.title('Area of layout during multiple objectives')
plt.legend()

In [ ]:
# print(wt_xaep[0])
# print(wt_yaep[0])
# for i in range(len(wt_xaep)):
#     wt_x = wt_xaep[i]
#     wt_y = wt_yaep[i]
#     aep_indv = get_ind_aep(wt_x,wt_y)
#     print(aep_indv) 

sim_res = windFarmModel(wt_xaep,wt_yaep)
plt.figure()
aep = sim_res.aep()
plt.scatter(wt_xaep,wt_yaep,facecolors = 'none')
c =plt.scatter(wt_xaep,wt_yaep, c=aep.sum(['wd','ws']))
plt.title('AEP optimization, AEP: %s [GWh]' %aep1 + ', Area: %s [Km2]' %area1, fontsize = 12)
plt.colorbar(c, label='AEP [GWh]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(wt_xaep,wt_yaep,color='b',label='AEP optimal')
plt.scatter(wt_xarea,wt_yarea,color='g',label='Area optimal')
#plt.scatter(wt_xload,wt_yload,color='m',label='Load constrain')
plt.scatter(wt_xcd,wt_ycd,color='k',label='EDenisty optimal')
plt.title('Wind Farm layout with 16turbines')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6.4, 4), dpi=80)
# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
plt.scatter(x = wt_xaep, y = wt_yaep,color = 'b' ) # c = single_AEP_OPT_GWh, cmap="rainbow"
#plt.clim(min_plot,max_plot)
# cbar = plt.colorbar(orientation="vertical")
# cbar.set_label(label="Single AEP GWh [GWh]", size = 15)
# cbar.ax.tick_params(labelsize=15)
plt.title('AEP optimization, AEP: %s [GWh]' %aep1 + ', Area: %s [Km2]' %area1, fontsize = 12)
# plt.savefig('./Figures/Optimized/ ' + 'AEP_LILLGRUND_HH'+ '_Opt_andr' + '.png')
plt.show()

In [ ]:
sim_res = windFarmModel(wt_aepx_opt,wt_aepy_opt)
plt.figure()
aep = sim_res.aep()
plt.scatter(wt_aepx_opt,wt_aepy_opt,facecolors = 'none')
c =plt.scatter(wt_aepx_opt,wt_aepy_opt, c=aep.sum(['wd','ws']))
plt.title('AEP optimization, AEP: %s [GWh]' %aep1 + ', Area: %s [Km2]' %area1, fontsize = 12)
plt.colorbar(c, label='AEP [GWh]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')

In [ ]:
areaopt = get_area(state_ar)
areaopt  = np.array(areaopt)
aep2 = np.round(areaopt[1],2)
area2 = np.round(areaopt[0]*1e-6,2)
area_opt = get_loads(state_ar)
area_opt_loads  = area_opt[2]
area_opt_tbfa = area_opt_loads[2]
print(area_opt_tbfa)

In [ ]:
plt.figure(figsize=(6.4, 4), dpi=80)
plt.scatter(x = wt_xarea, y = wt_yarea, color = 'g')
plt.title('Area optimization, AEP: %s [GWh]' %aep2 + ', Area: %s [Km2]' %area2, fontsize = 12)
plt.show()

In [ ]:
sim_res = windFarmModel(wt_areax_opt,wt_areay_opt)
plt.figure()
aep = sim_res.aep()
plt.scatter(wt_areax_opt,wt_areay_opt,facecolors = 'none')
c =plt.scatter(wt_areax_opt,wt_areay_opt, c=aep.sum(['wd','ws']))
plt.title('Area optimization, AEP: %s [GWh]' %aep2 + ', Area: %s [Km2]' %area2, fontsize = 12)
plt.colorbar(c, label='AEP [GWh]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')

In [ ]:
load_opt = get_loads(state_lc)
print(load_opt)
area_load = round(load_opt[1]*1e-6,2)
aep_load = np.array(load_opt[0])
aep_load = np.round(aep_load,2)
load_opt_loads = load_opt[2]
load_opt_tbfa = load_opt_loads[2] 
print(load_opt_tbfa)

In [ ]:
plt.figure(figsize=(6.4, 4), dpi=80)
plt.scatter(x = wt_xload, y = wt_yload, color = 'm')
plt.title('AEP_opt with loads, AEP: %s [GWh]' %aep_load + ', Area: %s [Km2]' %area2, fontsize = 11)
plt.show()

In [ ]:
sim_res = windFarmModel(wt_aepldx_opt,wt_aepldy_opt)
plt.figure()
aep = sim_res.aep()
plt.scatter(wt_aepldx_opt,wt_aepldy_opt,facecolors = 'none')
c =plt.scatter(wt_aepldx_opt,wt_aepldy_opt, c=aep.sum(['wd','ws']))
plt.title('AEP_opt with loads, AEP: %s [GWh]' %aep_load + ', Area: %s [Km2]' %area2, fontsize = 11)
plt.colorbar(c, label='AEP [GWh]')

In [ ]:
cd_opt = get_cap_density(state_cd)
print(cd_opt)
area_cd = round(cd_opt[2],2)
aep_cd = np.array(cd_opt[1])
aep_cd = np.round(aep_cd*1e-3,2)
print(area_cd)

In [ ]:
plt.figure(figsize=(6.4, 4), dpi=80)
plt.scatter(x = wt_xcd, y = wt_ycd,color = 'k')
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.show()

In [ ]:
sim_res = windFarmModel(wt_xcd,wt_ycd)
plt.figure()
aep = sim_res.aep()
plt.scatter(wt_xcd,wt_ycd,facecolors = 'none')
c =plt.scatter(wt_xcd,wt_ycd, c=aep.sum(['wd','ws']))
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.colorbar(c, label='AEP [GWh]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')

In [ ]:
plt.plot([0,1,2,3],[aep1,aep2,aep_load,aep_cd])
plt.scatter(x=0,y=aep1,color='b',label = 'AEP opt')
plt.scatter(x=1,y=aep2,color='g',label = 'Area opt')
plt.scatter(x=2,y=aep_load,color='m',label = 'AEP_opt with loads')
plt.scatter(x=3,y=aep_cd,color='k',label = 'Energy denisty opt')
#plt.xlabel('Optimization')
plt.ylabel('AEP [GWh]')
plt.xticks(np.arange(4),['AEP','Area','Loads','Energy_den'])
plt.legend()
plt.title('AEP during different optimizations')

In [ ]:
plt.plot([0,1,2,3],[area1,area2,area_load,area_cd])
plt.scatter(x=0,y=area1,color='b',label = 'AEP opt')
plt.scatter(x=1,y=area2,color='g',label = 'Area opt')
plt.scatter(x=2,y=area_load,color='m',label = 'AEP_opt with loads')
plt.scatter(x=3,y=area_cd,color='k',label = 'Energy denisty opt')
#plt.xlabel('Optimization')
plt.ylabel('Area [Km2]')
plt.xticks(np.arange(4),['AEP','Area','Loads','Energy_den'])
plt.legend()
plt.title('Area during different optimizations')

In [ ]:
plt.plot(recorder1['aep_final'],label='AEP optimization')
plt.plot(recorder2['aep_final'][:85],label='Area optimization')
plt.plot(recorder3['aep_final'],label='AEP loads opt')
plt.plot(recorder4['aep_final']*1e-3,label='Energy_density_opt')
plt.xlabel('No of iterations')
plt.ylabel('AEP in Gwh')
plt.title('Optimal AEP from sequential optimizations')
plt.legend()
plt.show()

In [ ]:
aep_area,aep=get_area(state)
print(aep_area)

In [ ]:
plt.plot(recorder2['final_area'][:85],'m',label='Area optimization')
plt.plot(recorder3['final_area'],label='AEP loads opt')
plt.plot(recorder4['final_area']*1e6,label='Energy_density_opt')
plt.plot(plt.xlim(),[aep_area,aep_area],label='AEP optimization')
plt.xlabel('No of iterations')
plt.ylabel('Area in m2')
plt.title('Optimal Area from sequential optimizations')
plt.legend()
plt.show()

In [ ]:
# for constraint_value, max_value in zip(problem4.driver.get_constraint_values['cost_comp.loads'], max_loads.ravel()):
#     print(constraint_value / max_value)
# ldel = loads[:,n,wt]/[max_loads[n, wt], max_loads[n, wt]]
# print(ldel)
loads_cd = cd_opt[3]
loads_cd_flap  = loads_cd[0]
loads_cd_edge  = loads_cd[1]
loads_cd_tbfa = loads_cd[2]
loads_cd_tbss = loads_cd[3]
loads_cd_tor  = loads_cd[4]

In [ ]:
flap_cd_norm = preprocessing.normalize([loads_cd_flap])
annotations = np.linspace(0,15,16,dtype = int)
plt.figure(figsize=(6,4))
plt.scatter(wt_xcd,wt_ycd,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xcd,wt_ycd, c=flap_cd_norm)
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.colorbar(c, label='Flapwise BM [kN-m]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xcd[i], wt_ycd[i]))
plt.legend()

In [ ]:
tbfa_cd_norm = preprocessing.normalize([loads_cd_tbfa])
annotations = np.linspace(0,15,16,dtype = int)
plt.figure(figsize=(6,4))
plt.scatter(wt_xcd,wt_ycd,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xcd,wt_ycd, c=tbfa_cd_norm)
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.colorbar(c, label='Tower Base Fore-aft [N/m2]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xcd[i], wt_ycd[i]))
plt.legend()

In [ ]:
tbss_cd_norm = preprocessing.normalize([loads_cd_tbss])
annotations = np.linspace(0,15,16,dtype = int)
plt.figure(figsize=(6,4))
plt.scatter(wt_xcd,wt_ycd,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xcd,wt_ycd, c=tbss_cd_norm)
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.colorbar(c, label='Tower Base Side-Side [N/m2]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xcd[i], wt_ycd[i]))
plt.legend()

In [ ]:
edge_cd_norm = preprocessing.normalize([loads_cd_edge])
annotations = np.linspace(0,15,16,dtype = int)
plt.figure(figsize=(6,4))
plt.scatter(wt_xcd,wt_ycd,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xcd,wt_ycd, c=edge_cd_norm)
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.colorbar(c, label='Edge-wise BM [N/m2]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xcd[i], wt_ycd[i]))
plt.legend()

In [ ]:
tor_cd_norm = preprocessing.normalize([loads_cd_tor])
annotations = np.linspace(0,15,16,dtype = int)
plt.figure(figsize=(6,4))
plt.scatter(wt_xcd,wt_ycd,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xcd,wt_ycd, c=tor_cd_norm)
plt.title('Energy_density_opt, AEP: %s [GWh]' %aep_cd + ', Area: %s [Km2]' %area_cd, fontsize = 11)
plt.colorbar(c, label='Shaft Torsion [N/m2]')
plt.xlabel('X[m]')
plt.ylabel('Y[m]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xcd[i], wt_ycd[i]))
plt.legend()

In [ ]:
flap_load_norm = preprocessing.normalize([load_opt_flap])
plt.figure(figsize=(6,4))
plt.scatter(wt_xload,wt_yload,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xload,wt_yload, c=tbfa_load_norm)
plt.title('AEP_opt with loads, AEP: %s [GWh]' %aep_load + ', Area: %s [Km2]' %area_load, fontsize = 11)
plt.colorbar(c, label='Tower Base Fore-aft [N/m2]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xload[i], wt_yload[i]))
plt.legend()

In [ ]:
tbfa_area_norm = preprocessing.normalize([area_opt_tbfa])
plt.figure(figsize=(6,4))
plt.scatter(wt_xarea,wt_yarea,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xarea,wt_yarea, c=tbfa_area_norm)
plt.title('Area_optimization, AEP: %s [GWh]' %aep2 + ', Area: %s [Km2]' %area2, fontsize = 11)
plt.colorbar(c, label='Tower Base Fore-aft [N/m2]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xarea[i], wt_yarea[i]))
plt.legend()

In [ ]:
tbfa_aep_norm = preprocessing.normalize([aep_opt_tbfa])
plt.figure(figsize=(6,4))
plt.scatter(wt_xaep,wt_yaep,facecolors = 'none',label = 'IEA34')
c =plt.scatter(wt_xaep,wt_yaep, c=tbfa_aep_norm)
plt.title('AEP_optimization, AEP: %s [GWh]' %aep1 + ', Area: %s [Km2]' %area1, fontsize = 11)
plt.colorbar(c, label='Tower Base Fore-aft [N/m2]')
for i, label in enumerate(annotations):
    plt.annotate(label, (wt_xaep[i], wt_yaep[i]))
plt.legend()

In [ ]:
plt.figure()
flow_map = sim_res.flow_map(ws = 10, wd=270)
plt.figure(figsize=(8,6))
flow_map.plot_wake_map()

In [ ]:
#resl = np.empty((5,1))
ws_l  = np.arange(3,10)
wdir = np.arange(225,270,10)
#ws_loads = np.empty((5,1))
resl = windFarmModel(wt_xaep,wt_yaep,ws=np.arange(10,16))
print(resl['TI_eff'])
resl.loads('OneWT')['LDEL'].values

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Make data.
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
# Customize the z axis.
ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
# A StrMethodFormatter is used automatically
ax.zaxis.set_major_formatter('{x:.02f}')

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()